<a href="https://colab.research.google.com/github/tsm-mehmetakiftasoz/tsm_makif/blob/main/Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
YOUR_API_KEY_HERE=

In [1]:
# Необходимые библиотеки
!pip install -q google-genai ipywidgets
import google.generativeai as genai
import ipywidgets as widgets
from IPython.display import display, HTML
from google.colab import output
import time, re

# 🔑 Настройка API
genai.configure(api_key="YOUR_API_KEY_HERE")

# 🌐 Статус подключения
bağlantı_durumu = widgets.HTML("<b style='color:#ff9800;'>🔄 Подключение к Gemini...</b>")

# 🪄 Интерфейс
output_area = widgets.Output(layout={
    'border': '3px solid #d97e07',
    'border_radius': '10px',
    'padding': '10px',
    'background_color': '#fff8dc',
    'max_height': '400px',
    'overflow_y': 'auto'
})

metin_kutusu = widgets.Textarea(
    placeholder='💬 Введите сообщение... (Enter = новая строка, Ctrl+Enter = отправить, чтобы завершить чат напишите "mischief managed.")',
    layout=widgets.Layout(width='80%', height='80px', padding='5px')
)

gonder_butonu = widgets.Button(
    description='🚀 Отправить',
    button_style='success',
    layout=widgets.Layout(width='20%', height='80px', padding='5px')
)

ui = widgets.VBox([bağlantı_durumu, output_area, widgets.HBox([metin_kutusu, gonder_butonu])])
display(ui)

# 🔗 Подключение к Gemini
conversation_history = []
try:
    model = genai.GenerativeModel("gemini-2.5-flash")
    chat = model.start_chat(history=conversation_history)
    bağlantı_durumu.value = "<b style='color:#ff9800;'>🔄 Подключение к Gemini...</b>"
    time.sleep(2)
    bağlantı_durumu.value = "<b style='color:#4CAF50;'>✅ Успешное подключение к Gemini.</b><br><i>Я торжественно клянусь, что замышляю шалости.</i>"
except Exception as e:
    bağlantı_durumu.value = f"<b style='color:red;'>❌ Ошибка подключения: {e}</b>"

# 🧙‍♂️ Отображение сообщений (баллон, Markdown ** → <b>)
def mesaj_goster(kimden, metin):
    # Markdown **text** → <b>text</b>
    metin_html = re.sub(r'\*\*(.*?)\*\*', r'<b>\1</b>', metin)
    renk = '#30D5C8' if kimden == 'Вы' else '#f5e36c'
    hizalama = 'right' if kimden == 'Вы' else 'left'
    with output_area:
        display(HTML(f"""
        <div class='msg' style='text-align:{hizalama}; margin: 5px; opacity:1; transition:opacity 1.5s;'>
            <div style='display:inline-block; background:{renk}; border-radius:10px; padding:8px 12px; max-width:80%; white-space: pre-wrap;'>
                <b>{kimden}:</b> {metin_html}
            </div>
        </div>
        """))

# 🧹 Завершение чата
def sohbeti_kapat():
    metin_kutusu.disabled = True
    gonder_butonu.disabled = True
    bağlantı_durumu.value = "<b style='color:#9c27b0;'>✨Чат завершен. В противном случае зло может увидеть.. 👁️</b>"

# 💨 Эффект исчезновения сообщений
def fade_out_mesajlar():
    display(HTML("""
    <script>
    let msgs = document.querySelectorAll('.msg');
    msgs.forEach(m => m.style.transition = 'opacity 2s');
    setTimeout(() => {
        msgs.forEach(m => m.style.opacity = 0;
    }, 500);
    </script>
    """))

# 🚀 Отправка сообщений
def mesaj_gonder(_=None):
    kullanici_mesaji = metin_kutusu.value.strip()
    if not kullanici_mesaji:
        return

    mesaj_goster("Вы", kullanici_mesaji)
    metin_kutusu.value = ''

    # Завершение чата
    if kullanici_mesaji.lower() == "mischief managed.":
        fade_out_mesajlar()
        sohbeti_kapat()
        metin_kutusu.placeholder = 'Чат завершен. В противном случае зло может увидеть 👁️'
        time.sleep(3.5)
        output_area.clear_output()
        metin_kutusu.placeholder = ' '
        return

    try:
        conversation_history.append({"role": "user", "content": kullanici_mesaji})
        yanit = chat.send_message(kullanici_mesaji)
        text_response = yanit.text
        mesaj_goster("Gemini", text_response)
        conversation_history.append({"role": "assistant", "content": text_response})
    except Exception as e:
        mesaj_goster("Система", f"Ошибка: {e}")

# 🎯 Привязка события к кнопке
gonder_butonu.on_click(mesaj_gonder)

# 🎹 Ctrl + Enter = Отправка
def js_klavye_dinleyici():
    display(HTML("""
    <script>
    document.querySelector('textarea').addEventListener('keydown', function(e) {
        if (e.ctrlKey && e.key === 'Enter') {
            google.colab.kernel.invokeFunction('notebook.mesajGonder', [], {});
            e.preventDefault();
        }
    });
    </script>
    """))
output.register_callback('notebook.mesajGonder', mesaj_gonder)
js_klavye_dinleyici()




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.5 MB/s eta 0:00:00
